In [1]:
# Dependencies
import requests
import json
import pandas as pd
# Google developer API key
from config import gkey

In [2]:
geocoding_df = pd.read_excel('data-files/Meetings-Events Test.xlsx', sheet_name='Meetings-Events Test')
geocoding_df.head()

,Event Name,Event Start Date,Event End Date,Location,Event Street Address,Event city,Event State or Province,Zip or Postal Code,Event Country,Event type,Notes
0,ACS Regional Midwest,2019-10-18,2019-10-20,"1450 Jayhawk Blvd Lawrence, KS",1450 Jayhawk Blvd,Lawrence,KS,66045.0,USA,Regional,NaN
1,ACS Regional Southwest,2019-10-29,2019-10-31,"2500 Broadway Lubbock, TX",2500 Broadway,Lubbock,TX,79409.0,USA,Regional,NaN
2,ChemEd,2020-04-10,2020-04-13,"77 Massachusetts Ave Cambridge, MA",77 Massachusetts Ave,Cambridge,MA,2139.0,USA,Specialty,NaN
3,ChemEd,2020-08-10,2020-08-13,"450 Serra Mall Stanford, CA",450 Serra Mall,Stanford,CA,94305.0,USA,Specialty,NaN
4,Leadership Institute,2019-12-10,2019-12-13,"1155 16th Street NW Washington, DC",1155 16th Street NW,Washington,DC,20036.0,USA,M&SS Event,NaN


In [13]:
for key, row in geocoding_df.iterrows():
    try:
        meeting_location = row['Location']

        target_search = meeting_location

        params = {
            "key": gkey,
            "input": target_search,
            "inputtype": 'textquery'
        }

        base_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"
        # base_url = 'https://maps.googleapis.com/maps/api/place/textsearch/json'    


        response = requests.get(base_url, params=params)
        places_data = response.json()    

        placeid = places_data['candidates'][0]['place_id']

        ## look up placeid
        params = {
            "key": gkey,
            "placeid": placeid,
        }

        base_url = "https://maps.googleapis.com/maps/api/place/details/json"

        # run a request using our params dictionary
        response = requests.get(base_url, params=params)

        # convert response to json
        places_data = response.json()

        place_name = places_data['result']['name']
        #place_address = places_data['result']['formatted_address']
        formatted_address = places_data['result']['formatted_address']
        place_id = placeid
        lat = places_data['result']['geometry']['location']['lat']
        lng = places_data['result']['geometry']['location']['lng']

        geocoding_df.loc[key,'name'] = place_name
        #geocoding_df.loc[key,'address'] = place_address
        geocoding_df.loc[key,'full_address'] = formatted_address
        geocoding_df.loc[key,'placeid'] = placeid
        geocoding_df.loc[key,'lat'] = lat
        geocoding_df.loc[key,'lng'] = lng

    except Exception as e:
        print(e)
        print(places_data)
        pass

In [14]:
geocoding_df.head()

,Event Name,Event Start Date,Event End Date,Location,Event Street Address,Event city,Event State or Province,Zip or Postal Code,Event Country,Event type,Notes,name,full_address,placeid,lat,lng
0,ACS Regional Midwest,2019-10-18,2019-10-20,"1450 Jayhawk Blvd Lawrence, KS",1450 Jayhawk Blvd,Lawrence,KS,66045.0,USA,Regional,NaN,Strong Hall,"Strong Hall, 1450 Jayhawk Blvd, Lawrence, KS 6...",ChIJrVdN_jVvv4cR0V8roW-lrAc,38.958542,-95.247614
1,ACS Regional Southwest,2019-10-29,2019-10-31,"2500 Broadway Lubbock, TX",2500 Broadway,Lubbock,TX,79409.0,USA,Regional,NaN,2500 Broadway St,"2500 Broadway St, Lubbock, TX 79409, USA",ChIJ9TVssrMS_oYRx7g7ZFTKqa0,33.583443,-101.874722
2,ChemEd,2020-04-10,2020-04-13,"77 Massachusetts Ave Cambridge, MA",77 Massachusetts Ave,Cambridge,MA,2139.0,USA,Specialty,NaN,77 Massachusetts Ave,"77 Massachusetts Ave, Cambridge, MA 02139, USA",ChIJJyW2xKlw44kRN8TOSYMuLAg,42.359244,-71.093139
3,ChemEd,2020-08-10,2020-08-13,"450 Serra Mall Stanford, CA",450 Serra Mall,Stanford,CA,94305.0,USA,Specialty,NaN,450 Serra Mall Parking,"77 Sweet Olive Way, Palo Alto, CA 94304, USA",ChIJD9T70TO7j4ARmIRa6Y_g4G0,37.440067,-122.169606
4,Leadership Institute,2019-12-10,2019-12-13,"1155 16th Street NW Washington, DC",1155 16th Street NW,Washington,DC,20036.0,USA,M&SS Event,NaN,1155 16th St NW,"1155 16th St NW, Washington, DC 20036, USA",ChIJYeienr-3t4kR_o9FcOBS0Eo,38.905236,-77.036045


In [15]:
geocoding_df.to_json('regional_events.json')